### 该类型特征的 column name 都以 CB_ 开头，意为Class and Brand（综合了车型和品牌）

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from xiao_utils import f

In [8]:
# 将level_id字段中的-替换为np.nan
df = pd.read_csv('../../data/origin/[new] yancheng_train_20171226.csv', dtype={'sale_date':str}, na_values=['-'], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'], format='%Y%m')

# 将price_level字段转换成有序类别的类型，并用其数值填入该列。
df['price_level'] = df['price_level'].astype('category', categories=['5WL','5-8W','8-10W','10-15W','15-20W','20-25W','25-35W','35-50W','50-75W'], ordered=True)
df['price_level'] = df['price_level'].cat.codes

# 待选方案：先把power和扭矩字段带/的行复制一份，然后将新行里的销量清零，将原行和新行的power和扭矩字段的值分别赋为slash前后的值。
# 现行方案：先他娘的直接把slash和后面的值删掉。省得影响记录条数相关的统计量。
def process_power_and_torque(s):
    return s.split('/')[0]
df['power'] = df['power'].astype(str).apply(process_power_and_torque).astype(float) #[18600]
df['engine_torque'] = df['engine_torque'].astype(str).apply(process_power_and_torque).astype(float)

# -------------------------------------------------------------
# 把2017年11月的数据拼进来，一块填入其特征，用于最终输出要提交的结果。
empty_Nov = pd.read_csv('../../data/origin/yancheng_testA_20171225.csv', dtype={'predict_date':str}, na_values=['-'], low_memory=False)
empty_Nov['predict_date']= pd.to_datetime(empty_Nov['predict_date'], format='%Y%m')
empty_Nov.rename(columns = {'predict_date': 'sale_date', 'predict_quantity':'sale_quantity'}, inplace = True)


# 读取玩了，先不急着拼，先把车型到品牌的映射关系join进来
class_to_brand = df[['class_id','brand_id']].groupby(['class_id']).mean().reset_index()
empyt_Nov = pd.merge(left=empty_Nov, right=class_to_brand, on='class_id', how='left')
empty_Nov['brand_id']= class_to_brand['brand_id']
# empty_Nov
# class_to_brand

# class_to_brand

# 读取完了，拼上去
df = pd.concat([df, empty_Nov])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20297 entries, 0 to 139
Data columns (total 32 columns):
TR                       20157 non-null object
brand_id                 20297 non-null int64
car_height               20157 non-null float64
car_length               20157 non-null float64
car_width                20157 non-null float64
class_id                 20297 non-null int64
compartment              20157 non-null float64
cylinder_number          20157 non-null float64
department_id            20157 non-null float64
displacement             20157 non-null float64
driven_type_id           20157 non-null float64
emission_standards_id    20157 non-null float64
engine_torque            20138 non-null float64
equipment_quality        20157 non-null float64
front_track              20157 non-null float64
fuel_type_id             20154 non-null float64
gearbox_type             20157 non-null object
if_MPV_id                20157 non-null float64
if_charging              20157 non-

In [13]:
cls_to_brand = df[['class_id','brand_id']].groupby(['class_id']).first().reset_index()

In [2]:
# 注意要把表中的 0 也都按空值 nan 处理，因为实际上不可能有真正有意义的 0 出现的！
df_C = pd.read_csv("../../data/features/C_features.csv", dtype={'sale_date':str}, na_values=['-',0], low_memory=False)
df_C['sale_date']= pd.to_datetime(df_C['sale_date'])
# df_C = df_C.drop('C_rcm_0', axis=1)

In [3]:
df_B = pd.read_csv("../../data/features/B_features.csv", dtype={'sale_date':str}, na_values=['-',0], low_memory=False)
df_B['sale_date']= pd.to_datetime(df_B['sale_date'])

In [4]:
df_C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5727 entries, 0 to 5726
Columns: 951 entries, class_id to C_rcm_rfdy_3
dtypes: datetime64[ns](1), float64(949), int64(1)
memory usage: 41.6 MB


In [5]:
df_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Columns: 507 entries, brand_id to B_rfdy_3
dtypes: datetime64[ns](1), float64(505), int64(1)
memory usage: 6.7 MB


In [6]:
df_C.head()

,class_id,sale_date,sale_quantity,C_som_1,C_som_2,C_som_3,C_som_4,C_som_5,C_som_6,C_som_7,...,C_rcm_sdy_3,C_rcm_sry_1,C_rcm_sry_2,C_rcm_sry_3,C_rcm_dfry_1,C_rcm_dfry_2,C_rcm_dfry_3,C_rcm_rfdy_1,C_rcm_rfdy_2,C_rcm_rfdy_3
0,103507,2015-03-01,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103507,2015-04-01,232.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103507,2015-05-01,226.0,232.0,58.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103507,2015-06-01,286.0,226.0,232.0,58.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,103507,2015-07-01,297.0,286.0,226.0,232.0,58.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_B.head()

,brand_id,sale_date,sale_quantity,B_som_1,B_som_2,B_som_3,B_som_4,B_som_5,B_som_6,B_som_7,...,B_sdy_3,B_sry_1,B_sry_2,B_sry_3,B_dfry_1,B_dfry_2,B_dfry_3,B_rfdy_1,B_rfdy_2,B_rfdy_3
0,12,2012-01-01,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12,2012-02-01,21.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,2012-03-01,10.0,21.0,47.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12,2012-04-01,10.0,10.0,21.0,47.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12,2012-05-01,10.0,10.0,10.0,21.0,47.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_C = pd.merge(df_C, cls_to_brand, how='left', on='class_id')

In [16]:
df_BC_tmp = pd.merge(df_C, df_B, how='left', on=['brand_id','sale_date'])

In [17]:
df_BC_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5727 entries, 0 to 5726
Columns: 1457 entries, class_id to B_rfdy_3
dtypes: datetime64[ns](1), float64(1454), int64(2)
memory usage: 63.7 MB


In [21]:
# 主要逻辑
def calc_ratio_class_divby_brand(df_BC_tmp, df_C):
    """
    Args:
        df_BC_tmp: 辅助
        df_C: 待扩充的目标表
    Return:
        tmp：基于历史销量上车型占品牌比重的信息，构造出的特征们
    """
    tmp = df_C

    # 过去几年内的每个月销量
    for i in range(62):
        c = df_BC_tmp['C_som_' + str(i+1)]
        b = df_BC_tmp['B_som_' + str(i+1)]
        tmp['CB_som_' + str(i+1)] = c / b


    # 该车型过去2~60个月分别的销量和
    for i in range(60):
        c = df_BC_tmp['C_ssm_' + str(i+2)]
        b = df_BC_tmp['B_ssm_' + str(i+2)]
        tmp['CB_ssm_' + str(i+2) ] = c / b
    
    # 一阶差分，一阶比值
    for i in range(61):
        c = df_BC_tmp['C_fd_' + str(i+1)]
        b = df_BC_tmp['B_fd_' + str(i+1)]
        tmp['CB_fd_' + str(i+1)] = c/b
        c = df_BC_tmp['C_fr_' + str(i+1)]
        b = df_BC_tmp['B_fr_' + str(i+1)]
        tmp['CB_fr_' + str(i+1)] = c/b
    
    # 二阶差分
    for i in range(60):
        c = df_BC_tmp['C_sd_' + str(i+1)]
        b = df_BC_tmp['B_sd_' + str(i+1)]
        tmp['CB_sd_' + str(i+1)] = c/b
        
    # 二阶比值
    for i in range(60):
        c = df_BC_tmp['C_sr_' + str(i+1)]
        b = df_BC_tmp['B_sr_' + str(i+1)]
        tmp['CB_sr_' + str(i+1)] = c/b
        
    # 比值的差分
    for i in range(60):
        c = df_BC_tmp['C_dfr_' + str(i+1)]
        b = df_BC_tmp['B_dfr_' + str(i+1)]
        tmp['CB_dfr_' + str(i+1)] = c/b
    
    # 差分的比值
    for i in range(60):
        c = df_BC_tmp['C_rfd_' + str(i+1)]
        b = df_BC_tmp['B_rfd_' + str(i+1)]
        tmp['CB_rfd_' + str(i+1)] = c/b
        
        
    # 相邻年，一阶差分，一阶比值
    for i in range(4):
        c = df_BC_tmp['C_fdy_' + str(i+1)]
        b = df_BC_tmp['B_fdy_' + str(i+1)]
        tmp['CB_fdy_' + str(i+1)] = c/b
        c = df_BC_tmp['C_fry_' + str(i+1)]
        b = df_BC_tmp['B_fry_' + str(i+1)]
        tmp['CB_fry_' + str(i+1)] = c/b
    
    # 相邻年，二阶差分
    for i in range(3):
        c = df_BC_tmp['C_sdy_' + str(i+1)]
        b = df_BC_tmp['B_sdy_' + str(i+1)]
        tmp['CB_sdy_' + str(i+1)] = c/b
        
    # 相邻年，二阶比值
    for i in range(3):
        c = df_BC_tmp['C_sry_' + str(i+1)]
        b = df_BC_tmp['B_sry_' + str(i+1)]
        tmp['CB_sry_' + str(i+1)] = c/b
        
    # 相邻年，比值的差分
    for i in range(3):
        c = df_BC_tmp['C_dfry_' + str(i+1)]
        b = df_BC_tmp['B_dfry_' + str(i+1)]
        tmp['CB_dfry_' + str(i+1)] = c/b
    
    # 相邻年，差分的比值
    for i in range(3):
        c = df_BC_tmp['C_rfdy_' + str(i+1)]
        b = df_BC_tmp['B_rfdy_' + str(i+1)]
        tmp['CB_rfdy_' + str(i+1)] = c/b
    

    
    # 注意要把np.inf替换为空值，在上面算月销量比例时，引入了inf，其实应该作为空值。
    # 注意过程中产生的 0 也要都换成空值！因为实际上不可能有有意义的0出现的。
    return tmp.replace([np.inf, -np.inf, 0], np.nan)

In [22]:
tmp = calc_ratio_class_divby_brand(df_BC_tmp, df_C)

In [23]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5727 entries, 0 to 5726
Columns: 1456 entries, class_id to CB_rfdy_3
dtypes: datetime64[ns](1), float64(1453), int64(2)
memory usage: 63.7 MB


In [24]:
df_BC_tmp.head()

,class_id,sale_date,sale_quantity_x,C_som_1,C_som_2,C_som_3,C_som_4,C_som_5,C_som_6,C_som_7,...,B_sdy_3,B_sry_1,B_sry_2,B_sry_3,B_dfry_1,B_dfry_2,B_dfry_3,B_rfdy_1,B_rfdy_2,B_rfdy_3
0,103507,2015-03-01,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2370.0,0.803762,NaN,NaN,-0.297090,NaN,NaN,0.638752,0.513924,NaN
1,103507,2015-04-01,232.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2434.0,0.724082,NaN,NaN,-0.423400,NaN,NaN,0.318985,0.534511,NaN
2,103507,2015-05-01,226.0,232.0,58.0,NaN,NaN,NaN,NaN,NaN,...,2486.0,0.463462,NaN,NaN,-1.016314,NaN,NaN,-0.258659,0.894208,NaN
3,103507,2015-06-01,286.0,226.0,232.0,58.0,NaN,NaN,NaN,NaN,...,2180.0,0.694375,NaN,NaN,-0.473018,NaN,NaN,0.211055,0.547706,NaN
4,103507,2015-07-01,297.0,286.0,226.0,232.0,58.0,NaN,NaN,NaN,...,2634.0,0.804476,NaN,NaN,-0.294399,NaN,NaN,0.629129,0.505695,NaN


In [25]:
tmp.head()

,class_id,sale_date,sale_quantity,C_som_1,C_som_2,C_som_3,C_som_4,C_som_5,C_som_6,C_som_7,...,CB_sdy_3,CB_sry_1,CB_sry_2,CB_sry_3,CB_dfry_1,CB_dfry_2,CB_dfry_3,CB_rfdy_1,CB_rfdy_2,CB_rfdy_3
0,103507,2015-03-01,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103507,2015-04-01,232.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103507,2015-05-01,226.0,232.0,58.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103507,2015-06-01,286.0,226.0,232.0,58.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,103507,2015-07-01,297.0,286.0,226.0,232.0,58.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# 存盘
tmp.to_csv("../../data/features/CB_features.csv",index=False)